In [47]:
(defclass polynomial ()
  ((poly :accessor poly
         :initform nil
         :initarg :poly)
   (deg :accessor poly-deg
        :initform 0
        :initarg :deg)
   )
)

#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>

In [48]:
;;; Make a polynomial taking advantage of program = data
;;; new polynomial term of the form a * x^b:
(defun make-polynomial (a b)
;;     (print (append b (cons * a)))
    (MAKE-INSTANCE 'polynomial :poly  `(+ (* ,a (expt x ,b))) :deg b)
;;     (MAKE-INSTANCE 'polynomial :poly  '(+ (* -2 (expt x 2)) ) :deg 1)
)

REDEFINITION-WITH-DEFUN: 
  redefining CL-JUPYTER-USER::MAKE-POLYNOMIAL in DEFUN


MAKE-POLYNOMIAL

In [49]:
;;;; TEST of make-polynomial
(let ((t1 (make-polynomial 2 3))
     (t2 (make-polynomial 0 0))
      )
     (print t1)
     (print t2)
)


2X^3 
0 

0

In [50]:
(defmethod degree ((p polynomial))
    (labels ( (degree-1 (poly max)
;;                         (print poly)
;;                         (format t "~C max: ~d calc: ~d " #\linefeed max (caddr (caddar poly)) )
                (cond  ( (null poly) max ) 
                        ( (> (caddr (caddar poly)) max) (degree-1 (cdr poly) (caddr (caddar poly))) )
                        (t (degree-1 (cdr poly) max))
                )
               )
             )
            (degree-1 (cdr (slot-value p 'poly)) 0)
     )
    
    
    )

REDEFINITION-WITH-DEFMETHOD: 
  redefining DEGREE (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::DEGREE (POLYNOMIAL) {1005D944F3}>

In [51]:
;;; TESTs of degree
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 7)) (* 3 (expt x 3))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 10)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
      )
     ( print (degree t1))
     ( print (degree t2))
     ( print (degree t3))
     
     )


3 
7 
10 

10

In [198]:
(defmethod print-object ((poly polynomial) out)
    (labels ((to-string (stack term poly  )
;;                 (print "------------------------------------")
;;                 (print stack)
;;                 (print  term)
;;                 (print poly)
                (cond  ((null term) (to-string-1  stack "" ))
                       ((atom term) term)
                       ((eql (car term) '*) (to-string (push `(,(to-string stack (cadr term) poly) ,(to-string stack (caddr term) poly) ) stack) (car poly) (cdr poly) ) )
                       ((eql (car term) 'expt) `(,(cadr term),(caddr term)))
;;                        (t (to-string-1 stack))
                 )

              )
              (to-string-1 (stack res )
;;                            (print "AT to-string-1")
;;                            (print stack)
;;                            (format t "res: ~s" res)
;;                            (print (car stack))
                           (let* ((coef (caar stack))
                                 (var  (caadar stack))
                                 (exp  (car (cdadar stack)))
                                 (str  (if (or (null coef) (equal res "") ) (if ( null coef ) res (format nil "~:[-~;~]" (>= coef 0)) ) (format nil "~a ~:[- ~;+ ~]" res (>= coef 0))))
                                 )
;;                                  (format t "term: ~a~a~a~C" coef var exp #\linefeed)
;;                                  (when (not (null coef)) (format res "~a ~:[ - ~; + ~]" res (> coef 0)))
                                 (cond 
                                       ((null stack) (if (equal res "") "0" res ));(format t "stack nil "))
;;                                        ((= coef exp 0) (to-string-1 (cdr stack) res))
                                       ((= coef 0) (to-string-1 (cdr stack) (format nil "~a" res)))
                                       ((= exp 0)  (to-string-1 (cdr stack) (format nil "~a~a" str (abs coef) )))
                                       ((and (= exp 1) (= (abs coef) 1)) (to-string-1 (cdr stack) (format nil "~a~a" str var )))
                                       ((= (abs coef) 1) (to-string-1 (cdr stack) (format nil "~a~a^~a" str var exp ) ) )
                                       ((= exp 1) (to-string-1 (cdr stack) (format nil "~a~a~a" str (abs coef) var)))
                                       (t (to-string-1 (cdr stack) (format nil "~a~a~a^~a" str (abs coef) var exp )))
                                )
                         )
             ) )
           (format out "~a" (to-string '() (cadr (slot-value poly 'poly)) (cddr (slot-value poly 'poly) )  ) )
     )
    
    

      
    )

REDEFINITION-WITH-DEFMETHOD: 
  redefining PRINT-OBJECT (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                           #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD


#<STANDARD-METHOD COMMON-LISP:PRINT-OBJECT (POLYNOMIAL T) {1003FA1293}>

In [93]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 0 (expt x 0)) ) :deg 1))  
      )
     (print t1)
     (print t2)
     (print t3)
     
     )


3X^3 + 2X^2 + X 
2X^7 + 3X^3 + X 
0 

0

In [94]:
(defmethod plus ((p1 polynomial) (p2 polynomial))
    
    (labels (
             (plus-1 (p1 p2 res)
                   (let ( (coef1 (cadar p1))
                          (var1  (cadr(caddar p1)))
                          (expt1 (caddr (caddar p1)))
                          (coef2 (cadar p2))
                          (var2  (cadr(caddar p2)))
                          (expt2 (caddr (caddar p2)))
                         )
;;                         (print "-----------------------------------")
;;                         (print res)
;;                         (print p1)
;;                         (print p2)
;;                         (format t "term1: ~a~a^~a" coef1 var1 expt1)
;;                         (format t "term2: ~a~a^~a" coef2 var2 expt2)
                        
                        (cond ((and (null p1) (null p2)) (cons '+ (reverse res)) )
                              ((null p1) (plus-1 nil (cdr p2) (push `(* ,coef2 (expt ,var2 ,expt2)) res)))
                              ((null p2) (plus-1 nil (cdr p1) (push `(* ,coef1 (expt ,var1 ,expt1)) res)))
                              ((= expt1 expt2)  
                                (plus-1 (cdr p1) (cdr p2) (push `(* ,(+ coef1 coef2) (expt ,var1 ,expt1)) res ) ))
                              ((< expt1 expt2) 
                                (plus-1 (cdr p1) p2  (push `(* ,coef1 (expt ,var1 ,expt1)) res)) )
                              ((> expt1 expt2) 
                                (plus-1 p1 (cdr p2)  (push `(* ,coef2 (expt ,var2 ,expt2)) res)) )
                         )
                    )
               ) 
             
             )
            (let ((poly_res (MAKE-INSTANCE 'polynomial ) )
                  (poly (plus-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() ))
                   
                   )
;;                  (print "after: ")
;;                  (print poly)
                 (setf (poly poly_res) poly)
                 (setf (poly-deg  poly_res) (degree poly_res))
;;                  (print (poly poly_res))
                 poly_res
                ) 
;;             (degree poly_res)
            
      )
;;         (print p1)
;;         (print p2)
    
    
)

REDEFINITION-WITH-DEFMETHOD: 
  redefining PLUS (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                   #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::PLUS (POLYNOMIAL POLYNOMIAL) {1004998BE3}>

In [95]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (plus t1 t2))
     ( print (plus t2 t1))
     ( print (plus t1 t3))
     ( print (plus t3 t1))
     ( print (plus t2 t3))
     ( print (plus t3 t2))
     
     )


2X^7 + 6X^3 + 2X^2 + 2X 
2X^7 + 6X^3 + 2X^2 + 2X 
X^10 + 6X^3 + 4X^2 + X 
X^10 + 6X^3 + 4X^2 + X 
X^10 + 2X^7 + 6X^3 + 2X^2 + X 
X^10 + 2X^7 + 6X^3 + 2X^2 + X 

X^10 + 2X^7 + 6X^3 + 2X^2 + X

In [96]:
(defmethod minus ((p1 polynomial) (p2 polynomial))
    
    (labels ( (minus-1 (p1 p2 res)
                   (let ( (coef1 (cadar p1))
                          (var1  (cadr(caddar p1)))
                          (expt1 (caddr (caddar p1)))
                          (coef2 (cadar p2))
                          (var2  (cadr(caddar p2)))
                          (expt2 (caddr (caddar p2)))
                         )
;;                         (print "-----------------------------------")
;;                         (print res)
;;                         (print p1)
;;                         (print p2)
;;                         (format t "term1: ~a~a^~a" coef1 var1 expt1)
;;                         (format t "term2: ~a~a^~a" coef2 var2 expt2)
                        
                        (cond ((and (null p1) (null p2)) (cons '+ (reverse res)) )
                              ((null p1) (minus-1 nil (cdr p2) (push `(* ,(- coef2) (expt ,var2 ,expt2)) res)))
                              ((null p2) (minus-1 nil (cdr p1) (push `(* ,coef1 (expt ,var1 ,expt1)) res)))
                              ((= expt1 expt2)  
                                (minus-1 (cdr p1) (cdr p2) (push `(* ,(- coef1 coef2) (expt ,var1 ,expt1)) res ) ))
                              ((< expt1 expt2) 
                                (minus-1 (cdr p1) p2  (push `(* ,coef1 (expt ,var1 ,expt1)) res)) )
                              ((> expt1 expt2) 
                                (minus-1 p1 (cdr p2)  (push `(* ,(- coef2) (expt ,var2 ,expt2)) res)) )
                         )
                    )
               ) 
             
             )
            (let ((poly_res (MAKE-INSTANCE 'polynomial ) )
                  (poly (minus-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() ))
                   
                   )
;;                  (print "after: ")
;;                  (print poly)
                 (setf (poly poly_res) poly)
                 (setf (poly-deg  poly_res) (degree poly_res))
;;                  (print (poly poly_res))
                 poly_res
                ) 
;;             (degree poly_res)
            
      )
;;         (print p1)
;;         (print p2)
    
    
)

REDEFINITION-WITH-DEFMETHOD: 
  redefining MINUS (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                    #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::MINUS (POLYNOMIAL POLYNOMIAL) {1005607B33}>

In [97]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 2 (expt x 2)) (* 3 (expt x 3))) :deg 3)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 1 (expt x 1)) (* 3 (expt x 3)) (* 2 (expt x 7))) :deg 3)) 
       (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (minus t1 t2))
     ( print (minus t2 t1))
     ( print (minus t1 t3))
     ( print (minus t3 t1))
     ( print (minus t2 t3))
     ( print (minus t3 t2))
     
     )


-2X^7 + 2X^2 
2X^7 - 2X^2 
-X^10 + X 
X^10 - X 
-X^10 + 2X^7 - 2X^2 + X 
X^10 - 2X^7 + 2X^2 - X 

X^10 - 2X^7 + 2X^2 - X

In [193]:
(defun poly-shortener (p term p_aux ) ; p is a list, term is one term of p (not in p) and p_aux is the result 
                      (print "========================================")
                      (print p)
                      (print term)
                      (print p_aux)
                      (let* ((poly_aux1 (MAKE-INSTANCE 'polynomial :poly (cons '+ `(,term) )))
                            (res (plus p_aux poly_aux1)) 
                           )
                            (print poly_aux1)
                            (print p)
                           (if (null p) res (poly-shortener (cdr p) (car p) res) )
                      )
                
                      
             )

REDEFINITION-WITH-DEFUN: 
  redefining CL-JUPYTER-USER::POLY-SHORTENER in DEFUN


POLY-SHORTENER

In [175]:
(defmethod times ((p1 polynomial) (p2 polynomial))
    (labels ((times-1 (p1 p2 res) 
                  
                  (cond ((null p2) res)
                        (t  (times-1 p1 (cdr p2) (times-2 p1 (car p2) res)))
                         
;;                          (let* ((poly_aux (MAKE-INSTANCE 'polynomial ) )
;;                                  (poly_res (MAKE-INSTANCE 'polynomial ) )
;; ;;                                  (poly (times-1 p1 (cdr p2) (times-2 p1 (car p2) res)) )
;;                                  (poly (times-2 p1 (car p2) res)) 
                                 
;;                                    )
;;                                  (cdr (slot-value p1 'poly))
;;                                  (setf (poly poly_aux) poly)
;;                                  (setf (poly poly_aux) res)
;;                                  (times-1 p1 (cdr p2) (cdr (slot-value (plus poly_aux poly_res) 'poly)))
;;                                 ()
                            ) 
;;                  (setf (poly poly_res) poly)
;;                  (setf (poly-deg  poly_res) (degree poly_res))
;;                 )
;;                            (times-1 p1 (cdr p2) (times-2 p1 (car p2) res)))
                      
;;                    )
             )
            (times-2 (p1 term2 res)
                     (let ((coef1 (cadar p1))
                           (var1  (cadr(caddar p1)))
                           (expt1 (caddr (caddar p1)))
                           (coef2 (cadr term2))
                           (var2  (cadr (caddr term2)))
                           (expt2 (caddr(caddr term2)))
                          )
;;                            (print "-----------------------------------")
;;                            (print res)
;;                            (print p1)
;;                            (print term2)
;;                            (format t "term1: ~a~a^~a" coef1 var1 expt1)
;;                            (format t "term2: ~a~a^~a" coef2 var2 expt2)
            
                         (cond ((null p1) res)
                               (t (times-2 (cdr p1) term2 (cons `(* ,(* coef1 coef2) (expt ,var1 ,(+ expt1 expt2))) res)))
                          )
                      )
             )
             (times-3 (p term p_aux )
;;                       (print "========================================")
;;                       (print p)
;;                       (print term)
;;                       (print p_aux)
                      (let* ((poly_aux1 (MAKE-INSTANCE 'polynomial :poly (cons '+ `(,term) )))
                            (res (plus p_aux poly_aux1)) 
                           )
;;                             (print poly_aux1)
                           (if (null p) res (times-3 (cdr p) (car p) res) )
                      )
                
                      
             )
            
            )
            (let ( (res_tmp (times-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() ))
;;                   (poly_aux1 (MAKE-INSTANCE 'polynomial 
;;                                 :poly (times-1 (cdr (slot-value p1 'poly)) (cdr (slot-value p2 'poly)) '() )))
                  )
;;                  (print "going into times-3")
                 (poly-shortener (cdr res_tmp) (car res_tmp) (make-polynomial 0 0)  )
             )
            )
    
    
    )

SIMPLE-STYLE-WARNING: 
  The variable VAR2 is defined but never used.
REDEFINITION-WITH-DEFMETHOD: 
  redefining TIMES (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                    #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::TIMES (POLYNOMIAL POLYNOMIAL) {10047EFF93}>

In [176]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 4 (expt x 3)) (* 3 (expt x 2)) (* 2 (expt x 1)) (* 1 (expt x 0))) :deg 4)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 3 (expt x 2)) (* 5 (expt x 0)) ) :deg 2)) 
;;        (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (times t1 t2))

     
     )


12X^5 + 9X^4 + 26X^3 + 18X^2 + 10X + 5 

12X^5 + 9X^4 + 26X^3 + 18X^2 + 10X + 5

In [100]:
(defmethod compose ((p1 polynomial) (p2 polynomial))
    (labels ((compose-1 (p1 p2 res) ;; p1 is a list, p2 is a polynomial as is res
                  (print res)
                (cond ((null p1) res)
                      (t (compose-1 (cdr p1) p2 (plus (make-polynomial (cadar p1) 0) (times p2 res))))
                 )
             
              
             
             ) )
             (compose-1 (cdr  (slot-value p1 'poly)) p2 (make-polynomial 0 0))
     )
    
 )

REDEFINITION-WITH-DEFMETHOD: 
  redefining COMPOSE (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                      #<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::COMPOSE (POLYNOMIAL POLYNOMIAL) {100625D913}>

In [177]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 4 (expt x 3)) (* 3 (expt x 2)) (* 2 (expt x 1)) (* 1 (expt x 0))) :deg 4)) 
       (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 3 (expt x 2)) (* 5 (expt x 0)) ) :deg 2)) 
;;        (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (compose t1 t2))

     
     )


0 
4 
12X^2 + 23 
36X^4 + 129X^2 + 117 
108X^6 + 567X^4 + 996X^2 + 586 
108X^6 + 567X^4 + 996X^2 + 586 

108X^6 + 567X^4 + 996X^2 + 586

In [117]:
(defmethod evaluate ((p1 polynomial) val)
             (eval (subst val 'x (poly p1) ))
 )

REDEFINITION-WITH-DEFMETHOD: 
  redefining EVALUATE (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>
                       #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::EVALUATE (POLYNOMIAL T) {1003838E63}>

In [118]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 4 (expt x 3)) (* 3 (expt x 2)) (* 2 (expt x 1)) (* 1 (expt x 0))) :deg 4)) 
;;        (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 3 (expt x 2)) (* 5 (expt x 0)) ) :deg 2)) 
;;        (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
     ( print (evaluate t1 3))

     
     )


"eval" 
(+ (* 4 (EXPT 3 3)) (* 3 (EXPT 3 2)) (* 2 (EXPT 3 1)) (* 1 (EXPT 3 0))) 
142 

142

In [200]:
(defmethod differentiate ((p polynomial))
    (labels ((differentiate-1 (p1 res) ; p1 is the list
;;                               (print "-----------------------------------")
                           (print res)
;;                            (print p1)
             (let ((coef (cadar p1))
                    (var (cadr (caddar p1)))
                    (exp (caddr (caddar p1)))
                          )
;;                            
;;                            (print term2)
;;                            (format t "term1: ~a~a^~a" coef var exp)
;;                            (format t "term2: ~a~a^~a" coef2 var2 expt2)
            
                     (cond ((null p1) res)
                           ((= exp 0) (differentiate-1 (cdr p1) (cons `(* 0 (expt ,var 0)) res)))
                           (t (differentiate-1 (cdr p1) (cons `(* ,(* coef exp) (expt ,var ,(- exp 1))) res) ))
                          )
                      )
             
             )
            
         ) 
            (let ((aux (differentiate-1 (cdr (slot-value p 'poly)) '()) )
                     )
                 (print "fixing")
                 (print aux)
                     (MAKE-INSTANCE 'polynomial :poly 
                            (poly-shortener (cdr aux) (car aux) (make-polynomial 0 0) ))
;;                  (print "END")
                    )
            
;;                             (poly-shortener (cdr res_tmp) (car res_tmp) (make-polynomial 0 0)  )
    )
 )
    

REDEFINITION-WITH-DEFMETHOD: 
  redefining DIFFERENTIATE (#<STANDARD-CLASS CL-JUPYTER-USER::POLYNOMIAL>) in DEFMETHOD


#<STANDARD-METHOD CL-JUPYTER-USER::DIFFERENTIATE (POLYNOMIAL) {1004433163}>

In [ ]:
(let ( (t1 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 4 (expt x 3)) (* 3 (expt x 2)) (* 2 (expt x 1)) (* 1 (expt x 0))) :deg 4)) 
;;        (t2 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 3 (expt x 2)) (* 5 (expt x 0)) ) :deg 2)) 
;;        (t3 (MAKE-INSTANCE 'polynomial :poly  '(+ (* 2 (expt x 2)) (* 3 (expt x 3)) (* 1 (expt x 10))) :deg 3)) 
      )
;;      ( print (differentiate t1 ))
     (differentiate t1)
;;      ( print (differentiate (differentiate t1 )))

     
     )


NIL 
((* 12 (EXPT X 2))) 
((* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
((* 2 (EXPT X 0)) (* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
((* 0 (EXPT X 0)) (* 2 (EXPT X 0)) (* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
"fixing" 
((* 0 (EXPT X 0)) (* 2 (EXPT X 0)) (* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
"========================================" 
((* 2 (EXPT X 0)) (* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
(* 0 (EXPT X 0)) 
0 
0 
((* 2 (EXPT X 0)) (* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
"========================================" 
((* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
(* 2 (EXPT X 0)) 
0 
2 
((* 6 (EXPT X 1)) (* 12 (EXPT X 2))) 
"========================================" 
((* 12 (EXPT X 2))) 
(* 6 (EXPT X 1)) 
2 
6X 
((* 12 (EXPT X 2))) 
"========================================" 
NIL 
(* 12 (EXPT X 2)) 
6X + 2 
12X^2 
NIL 

In [159]:
(defvar *zero*)
(defvar *p1*)
(defvar *p2*)
(defvar *p3*)
(defvar *p4*)

(defvar *test*)

(setq *zero* (make-polynomial 0 0))
(setq *p1* (make-polynomial 4 3))
(setq *p2* (make-polynomial 3 2))
(setq *p3* (make-polynomial 1 0))
(setq *p4* (make-polynomial 2 1))

(setq *test* (make-polynomial 5 5))
;; (make-polynomial 4 3)

5X^5